# Formatting lice data (2005-2011) in SAS Studio

The data was in a format difficult to read. The first row - infomation about farmname, municipality, and locaiton. Then follows the information by columns - year, month, date and abundance of lice.
I used SAS (with help of SAS experts) to re-format the data. 
The code below re-arranges the data conditionally such that instead on line of text inbetween data rows, we have all the information stored in the variables (columns).


The peek into data file - the data is in hierarchical form:

Fishfarm:,null,1816,10451,"(-999.0,-999.0)",
2005,2,1,5.3,null,0.1,0.1
2005,3,1,4.8,null,0.2,0.1


### Indicating file path (same file is also stored at ftp)

In [ ]:
%let ohi_path=/folders/myshortcuts/ohi_raw;
libname ohi base "&ohi_path/SAS_data_mining/Output_files";
filename in "/folders/myshortcuts/ohi_raw/akva/lice_data_2005_2011.csv";

###  Creating a custom format - mynull. to replace all the "null" with "." 

In [ ]:
proc format;
invalue mynull 'null'=. ;
run;

### Re-formatting data 
Reformatting by using INPUT statement and IF THEN conditions

In [ ]:
data salmon;
 infile in dsd dlm=',';
 input @1 test : $30. @;
 if test ='Year' then delete;
 if test =' ' then delete;
 retain What Farmname Municipality Location Latlon; 
 informat Municipality Location mynull.;
 if test='Fishfarm:' then input @1 What : $9. Farmname Municipality Location Latlon : $40.; 
 else if input(test,??4.) ne . 
 then do; 
 informat Temperature Sitting_lice Moving_lice Adult_females  mynull.;
 input @1 Year Month Date Temperature Sitting_lice Moving_lice Adult_females  ;
 output;
 end;
 drop test;
run;

### Looking at the head of the table: 
Farmnames were unreadable, I removed them. Some dates are missing and sometimes there is missing data for some of the variables.

In [12]:
options obs=10;
proc print;
run;

Obs,What,Farmname,Municipality,Location,Latlon,Temperature,Sitting_lice,Moving_lice,Adult_females,Year,Month,Date
1,Fishfarm:,.,1816,10451,"(-999.0,-999.0)",5.30,.,0.10,0.10,2005,2,1
2,Fishfarm:,.,1816,10451,"(-999.0,-999.0)",4.80,.,0.20,0.10,2005,3,1
3,Fishfarm:,.,1816,10451,"(-999.0,-999.0)",5.35,.,0.00,0.20,2005,4,1
4,Fishfarm:,.,1816,10451,"(-999.0,-999.0)",6.09,.,0.15,0.15,2005,5,1
5,Fishfarm:,.,1816,10451,"(-999.0,-999.0)",7.61,.,1.33,1.33,2005,6,1
6,Fishfarm:,.,1816,10451,"(-999.0,-999.0)",8.73,.,0.00,0.00,2005,7,1
7,Fishfarm:,.,1816,10451,"(-999.0,-999.0)",10.62,.,0.00,0.00,2005,10,1
8,Fishfarm:,.,1816,10451,"(-999.0,-999.0)",10.21,.,0.05,0.05,2005,11,1


### Epxporting the sas table as csv

In [ ]:
proc export data=salmon
outfile="&ohi_path/akva/lice_vers2_2005_2011_formatted.csv";
run;